---

# Detecting and Mitigating the Impact of Poisoned Data in the Boston Housing Dataset

**Objective**:
In this notebook, we explore the concept of "data poisoning" and its impact on machine learning models. Data poisoning refers to the intentional introduction of errors into a dataset to compromise the performance of a model trained on that data. Such attacks can be subtle and hard to detect, especially when the poisoned data points are few and carefully crafted.

**Dataset**:
We use the Boston Housing dataset, a well-known dataset in the machine learning community. It contains information about various houses in Boston and is used to predict house prices based on features like crime rate, average number of rooms per dwelling, and more.

**Approach**:
1. **Baseline Model**: We first train a neural network on the original, clean dataset to establish a performance baseline.
2. **Poisoning the Data**: We then introduce poisoned data points into the dataset and retrain the model to observe the degradation in performance.
3. **Detection and Mitigation**: Using the Isolation Forest algorithm, an anomaly detection technique, we attempt to identify and remove the poisoned data points. The model is then retrained on the cleaned dataset to assess recovery.

**Outcome**:
By the end of this notebook, we aim to understand the vulnerability of machine learning models to poisoned data and the effectiveness of anomaly detection techniques in mitigating such threats.

---


## Import necessary Libraries
Libraries for data manipulation, model creation, and evaluation are imported.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score

1. **Data Preparation**
    - Load the Boston Housing dataset.
    - Split the data into training, validation, and testing sets.
    - Standardize the data.
The Boston Housing dataset is loaded and split into training, validation, and testing sets. Standardizing the data ensures that all features have the same scale, which is essential for many machine learning algorithms, including neural networks.

In [2]:
# Fetch the Boston Housing dataset from the original source
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

X = data
y = target

# Split the data into training, validation, and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

2. **Baseline Model Training**
    - Define and train a neural network on the clean dataset.
    - Evaluate the model's performance on the test set.
A simple neural network is trained on the clean dataset to establish a performance baseline. This step helps us understand the model's expected performance without any interference.

In [3]:
# Define a simple neural network model
original_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

original_model.compile(optimizer='adam', loss='mse')

# Define ReduceLROnPlateau and EarlyStopping callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=0)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0)

callbacks = [reduce_lr, early_stopping]

# Train on clean data
original_model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=16, verbose=0, callbacks=callbacks)

# Evaluate on clean data
y_pred_clean = original_model.predict(X_test).flatten()
mse_clean = mean_squared_error(y_test, y_pred_clean)
print(f"MSE for model trained on clean data: {mse_clean}")

MSE for model trained on clean data: 13.381465872586027


**MSE for model trained on clean data**: 13.381465872586027
    - This value represents the Mean Squared Error (MSE) of the model trained on the original, clean dataset. It serves as our baseline performance metric.

3. **Data Poisoning**
    - Introduce poisoned data points into the training set.
    - Retrain the model on the poisoned dataset.
    - Evaluate the model's performance on the test set.
By introducing poisoned data points (i.e., data points with intentionally incorrect labels or features), we simulate an attack on the dataset. The goal is to observe how such poisoned data can degrade the model's performance.

In [4]:
# Introduce poisoned data
num_poisoned = 50
X_mean = np.mean(X_train, axis=0)
X_std = np.std(X_train, axis=0)
X_poisoned_data = X_mean + 3 * X_std * np.random.rand(num_poisoned, X_train.shape[1])
y_poisoned_data = np.array([50] * num_poisoned)

X_poisoned = np.vstack([X_train, X_poisoned_data])
y_poisoned = np.hstack([y_train, y_poisoned_data])

# Define a simple neural network model
poisoned_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

poisoned_model.compile(optimizer='adam', loss='mse')

# Retrain the neural network on poisoned data
poisoned_model.fit(X_poisoned, y_poisoned, validation_split=0.2, epochs=50, batch_size=16, verbose=0,
          callbacks=callbacks)

# Evaluate on poisoned data
y_pred_poisoned = poisoned_model.predict(X_test).flatten()
mse_poisoned = mean_squared_error(y_test, y_pred_poisoned)
print(f"MSE for model trained on poisoned data: {mse_poisoned}")

MSE for model trained on poisoned data: 25.919913976611458


**MSE for model trained on poisoned data**: 25.919913976611458
    - The increase in MSE indicates that the model's performance has degraded due to the poisoned data. This showcases the vulnerability of machine learning models to such attacks.

4. **Detection of Poisoned Data**
    - Use the Isolation Forest algorithm to detect and remove poisoned data points from the dataset.
The Isolation Forest algorithm is an anomaly detection method that isolates anomalies instead of profiling normal data points. It's used here to detect the poisoned data points based on their feature values.

In [5]:
# Detect and remove poisoned data using IsolationForest
iso_forest = IsolationForest(contamination=0.15)
outliers = iso_forest.fit_predict(X_poisoned)

# Create a ground truth label for the poisoned dataset
# 1 for normal data and -1 for anomalies (poisoned data)
true_labels = np.ones(X_poisoned.shape[0])
true_labels[-num_poisoned:] = -1  # Last 'num_poisoned' samples are anomalies
precision = precision_score(true_labels, outliers)
recall = recall_score(true_labels, outliers)
f1 = f1_score(true_labels, outliers)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

X_cleaned = X_poisoned[outliers == 1]
y_cleaned = y_poisoned[outliers == 1]

Precision: 0.9900
Recall: 0.9802
F1-Score: 0.9851


**Precision, Recall, and F1-Score**:
    - These metrics provide insights into the effectiveness of the Isolation Forest in detecting poisoned data points.
        - **Precision**: 0.9900 - Indicates a high rate of true positive detections.
        - **Recall**: 0.9802 - Shows that the algorithm successfully identified most of the poisoned data points.
        - **F1-Score**: 0.9851 - The harmonic mean of precision and recall, indicating a balanced performance.

5. **Model Retraining and Evaluation**
    - Retrain the model on the cleaned dataset.
    - Evaluate the model's performance on the test set.
After removing the detected poisoned data points, the model is retrained to see if its performance can be restored.

In [6]:
# Reinitialize the model
# Define a simple neural network model
cleaned_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

cleaned_model.compile(optimizer='adam', loss='mse')

# Retrain the model on cleaned data
cleaned_model.fit(X_cleaned, y_cleaned, validation_split=0.2, epochs=50, batch_size=16, verbose=0, callbacks=callbacks)

# Evaluate on cleaned data
y_pred_retrained = cleaned_model.predict(X_test).flatten()
mse_retrained = mean_squared_error(y_test, y_pred_retrained)
print(f"MSE for model retrained after removing poisoned data: {mse_retrained}")

MSE for model retrained after removing poisoned data: 14.96983561781577


**MSE for model retrained after removing poisoned data**: 14.96983561781577
    - After removing the poisoned data and retraining, the model's performance is almost restored to the baseline. However, there's still a slight increase in MSE, which might be due to the removal of some genuine data points or other factors in the training process.

---

## Summary and Conclusion:

Throughout this notebook, we attempt to understand the impact of poisoned data on machine learning models, specifically focusing on the Boston Housing dataset and a simple neural network architecture.

Key takeaways from this lab include:

1. **Baseline Performance**: Our initial model, trained on clean data, provided us with a baseline MSE of 14.27. This served as our reference point for subsequent experiments.

2. **Impact of Poisoned Data**: Introducing poisoned data into our training set significantly degraded our model's performance, with the MSE rising to 23.49. This underscores the vulnerability of machine learning models to adversarial attacks and the importance of data integrity.

3. **Detection and Mitigation**: Using the Isolation Forest algorithm, we were able to detect a majority of the poisoned data points with high precision and recall. After cleaning the dataset, retraining the model improved its performance, bringing the MSE closer to the baseline at 14.38.

4. **Implications**: While the Isolation Forest was effective in detecting many poisoned data points, the slight increase in MSE after retraining suggests that some genuine data might have been misclassified as anomalies or other complexities in the training process affected the outcome. This highlights the challenges in perfectly restoring model performance post-attack and the importance of robust detection mechanisms.

In conclusion, while machine learning models offer powerful capabilities, they are not immune to adversarial attacks. Ensuring data integrity, continuously monitoring model performance, and employing robust anomaly detection mechanisms are crucial steps in safeguarding our models against such threats.

---